In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.preprocessing import OneHotEncoder

# Lade die Trainingsdaten
df = pd.read_csv('analysis/train_split_merged_data_updated.csv')
df['date'] = pd.to_datetime(df['date'])

# Filtere auf Warengruppe 4
df_g4 = df[df['warengruppe'] == 4].copy()

print('Datensatz Warengruppe 4:')
print(f'Zeilen: {len(df_g4)}')
print(f'\nFirst rows:')
print(df_g4[['date', 'umsatz', 'Temperatur']].head())

In [ ]:
# Erstelle Wochentag-Spalte
df_g4['Wochentag'] = df_g4['date'].dt.day_name()

# Übersicht über Wochentage
print('Wochentag-Verteilung:')
print(df_g4['Wochentag'].value_counts())
print('\nUmsatz nach Wochentag:')
print(df_g4.groupby('Wochentag')['umsatz'].agg(['count', 'mean', 'std']))

In [ ]:
# Visualisiere: Umsatz nach Wochentag
plt.figure(figsize=(12, 5))
wochentage_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
df_g4_grouped = df_g4.groupby('Wochentag')['umsatz'].mean().reindex(wochentage_order)
df_g4_grouped.plot(kind='bar', color='steelblue')
plt.title('Average Sales by Day of Week (Warengruppe 4)', fontsize=14)
plt.xlabel('Wochentag', fontsize=12)
plt.ylabel('Durchschnittlicher Umsatz (€)', fontsize=12)
plt.xticks(rotation=45)
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Vorbereite Daten: entferne NaN in Temperatur
data_clean = df_g4[['Temperatur', 'Wochentag', 'umsatz']].dropna()

print(f'Zeilen nach NaN-Entfernung: {len(data_clean)} (von {len(df_g4)})')
print(f'\nBasic stats:')
print(data_clean.describe())

# One-Hot Encoding für Wochentage (erstelle Dummy-Variablen)
# Verwende drop_first=True, um Multicollinearity zu vermeiden
wochentag_dummies = pd.get_dummies(data_clean['Wochentag'], drop_first=True, prefix='Day')

print(f'\nWochentag Dummy-Spalten:')
print(wochentag_dummies.head())

# Kombiniere Temperatur + Wochentag-Dummies
X = pd.concat([data_clean[['Temperatur']].reset_index(drop=True), 
               wochentag_dummies.reset_index(drop=True)], axis=1)
y = data_clean['umsatz'].reset_index(drop=True)

print(f'\nFinal X shape: {X.shape}')
print(f'Final y shape: {y.shape}')
print(f'\nFeature-Namen: {X.columns.tolist()}')

In [ ]:
# Trainiere das Modell mit Temperatur + Wochentag
model = LinearRegression()
model.fit(X, y)

# Vorhersagen
y_pred = model.predict(X)

# Performance-Metriken
r2 = r2_score(y, y_pred)
rmse = np.sqrt(mean_squared_error(y, y_pred))
mae = mean_absolute_error(y, y_pred)

print('='*70)
print('MODEL PERFORMANCE: Temperatur + Wochentag (Warengruppe 4)')
print('='*70)
print(f'R² Score: {r2:.4f}')
print(f'RMSE: {rmse:.4f}€')
print(f'MAE: {mae:.4f}€')
print(f'\nInterpretation:')
print(f'- R² = {r2:.4f}: Das Modell erklärt {r2*100:.2f}% der Varianz')
print(f'- Durchschnittlicher Fehler (MAE): ±{mae:.2f}€')

In [ ]:
# Zeige die Modellgleichung
print('\n' + '='*70)
print('MODELLGLEICHUNG')
print('='*70)

intercept = model.intercept_
coef_temp = model.coef_[0]
coef_wochentag = model.coef_[1:]
feature_names = ['Temperatur'] + X.columns.tolist()[1:]

print(f'\nIntercept (Basis-Umsatz): {intercept:.4f}€')
print(f'\nKoeffizient für Temperatur: {coef_temp:.4f}€ pro °C')

print(f'\nKoeffizienten für Wochentage (relativ zu {X.columns.tolist()[1:][0]}):')
for fname, coef in zip(X.columns.tolist()[1:], coef_wochentag):
    print(f'  {fname:20s}: {coef:+.4f}€')

# Volle Gleichung ausgeben
print(f'\n' + '-'*70)
print('VEREINFACHTE GLEICHUNG:')
print('-'*70)
equation = f'umsatz = {intercept:.4f} + {coef_temp:.4f}×Temperatur'
for fname, coef in zip(X.columns.tolist()[1:], coef_wochentag):
    equation += f' {coef:+.4f}×{fname}'

print(f'{equation}')

In [ ]:
# Vergleich: nur Temperatur vs. Temperatur + Wochentag
print('\n' + '='*70)
print('VERGLEICH: NUR TEMPERATUR vs. TEMPERATUR + WOCHENTAG')
print('='*70)

# Modell nur mit Temperatur
X_temp_only = data_clean[['Temperatur']].values
model_temp_only = LinearRegression()
model_temp_only.fit(X_temp_only, y)
y_pred_temp = model_temp_only.predict(X_temp_only)
r2_temp = r2_score(y, y_pred_temp)
mae_temp = mean_absolute_error(y, y_pred_temp)

print(f'\nNur Temperatur:')
print(f'  R² = {r2_temp:.4f} ({r2_temp*100:.2f}%)')
print(f'  MAE = {mae_temp:.4f}€')

print(f'\nTemperatur + Wochentag:')
print(f'  R² = {r2:.4f} ({r2*100:.2f}%)')
print(f'  MAE = {mae:.4f}€')

improvement = ((r2 - r2_temp) / r2_temp) * 100
print(f'\n✓ Verbesserung: {improvement:+.2f}% durch Wochentag-Feature')
print(f'✓ MAE-Reduktion: {mae_temp - mae:.2f}€')

In [ ]:
# Visualisierung: Actual vs Predicted
plt.figure(figsize=(11, 6))
plt.scatter(y, y_pred, alpha=0.5, s=30)
plt.plot([y.min(), y.max()], [y.min(), y.max()], 'r--', lw=2, label='Perfect prediction')
plt.xlabel('Actual Umsatz (€)', fontsize=12)
plt.ylabel('Predicted Umsatz (€)', fontsize=12)
plt.title('Linear Regression: Actual vs Predicted (Temperatur + Wochentag)', fontsize=14)
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Residualplot
residuals = y - y_pred

plt.figure(figsize=(11, 6))
plt.scatter(y_pred, residuals, alpha=0.5, s=30)
plt.axhline(y=0, color='r', linestyle='--', lw=2)
plt.xlabel('Predicted Umsatz (€)', fontsize=12)
plt.ylabel('Residuals', fontsize=12)
plt.title('Residualplot: Temperatur + Wochentag', fontsize=14)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print('Residual Statistics:')
print(f'Mean: {residuals.mean():.6f} (sollte nahe 0 sein)')
print(f'Std: {residuals.std():.6f}')

In [ ]:
# Vorhersagen für verschiedene Szenarien
print('\n' + '='*70)
print('BEISPIEL-VORHERSAGEN: Verschiedene Tage + Temperaturen')
print('='*70)

wochentage_list = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
temperatures_list = [0, 10, 20, 30]

print('\nUmsatz-Vorhersage (€) für verschiedene Kombinationen:\n')
print('Temperatur'.ljust(15), end='')
for tag in wochentage_list:
    print(f'{tag[:3]:>8}', end='')
print()
print('-' * 75)

for temp in temperatures_list:
    print(f'{temp:>3}°C'.ljust(15), end='')
    for tag in wochentage_list:
        # Erstelle ein Feature-Vektor für diese Kombination
        feature_row = [temp]
        for col in X.columns[1:]:
            # Extrahiere den Wochentag aus dem Feature-Namen (z.B. 'Day_Monday')
            if tag in col:
                feature_row.append(1)
            else:
                feature_row.append(0)
        
        pred = model.predict([feature_row])[0]
        print(f'{pred:>8.2f}', end='')
    print()

In [ ]:
# Effektgröße: Durchschnittlicher Umsatz pro Wochentag
print('\n' + '='*70)
print('DURCHSCHNITTLICHE UMSÄTZE PRO WOCHENTAG (Rohdaten)')
print('='*70)

wochentag_means = df_g4.groupby('Wochentag')['umsatz'].mean().reindex(
    ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
)

for tag, mean in wochentag_means.items():
    print(f'{tag:12s}: {mean:7.2f}€')

# Finde Best & Worst Day
best_day = wochentag_means.idxmax()
worst_day = wochentag_means.idxmin()
diff = wochentag_means[best_day] - wochentag_means[worst_day]

print(f'\nBesser Tag: {best_day} ({wochentag_means[best_day]:.2f}€)')
print(f'Schlechtester Tag: {worst_day} ({wochentag_means[worst_day]:.2f}€)')
print(f'Differenz: {diff:.2f}€ ({(diff/wochentag_means[worst_day]*100):.1f}%)')